In [1]:
import pandas as pd
import csv

In [2]:
train = pd.read_csv('train.dat',sep='\t',header=None)
train = train.rename(columns = {0:'user',1:'item',2:'rating'})
test = pd.read_csv('test.dat',sep='\t',header=None)
test = test.rename(columns = {0:'user',1:'item',2:'rating'})
validation = pd.read_csv('valid.dat',sep='\t',header=None)
validation = validation.rename(columns = {0:'user',1:'item',2:'rating'})
df = pd.concat([train,test,validation])

In [ ]:
df.to_csv('ratings.csv',index=False)

In [3]:
len(df['item'].value_counts())

2680

In [4]:
jiba = pd.read_csv('i_map.dat',sep = '\t',header = None)
jiba

,0,1
0,0,1
1,1,2
2,2,3
3,3,7
4,4,9
...,...,...
2675,2675,8154
2676,2676,8163
2677,2677,8166
2678,2678,8168


In [5]:
haha = pd.read_csv('i2kg_map.tsv',sep='\t', header=None)
haha = haha.rename(columns = {0:'item',1:'name',2:'url'})
haha = haha.sort_values(by='item',ascending=True)

In [6]:
haha_final = haha[haha['item'].isin(jiba[1])]

In [9]:
with open("attribute.json", 'r', encoding='UTF-8') as f:
    info = json.load(f)
info = {int(k): v for k, v in info.items()}

In [10]:
from concurrent.futures import ProcessPoolExecutor
THREADS = 16
def map_aspect_values_to_movies(x):
    (film, meta), aspect = x
    aspects = dict()
    if aspect == "director" and type(meta[aspect]) is str:
        aspects[meta[aspect]] = 1
    else:
        for g in meta[aspect]:
            aspects[g] = 1
    return film, meta, aspects


def dict_movie_aspect(paper_films, aspect):
    paper_films_aspect_prepended = map(
        lambda e: (e, aspect), list(paper_films.items()))
    aspect_dict = dict()
    with ProcessPoolExecutor(max_workers=THREADS) as executor:
        results = executor.map(map_aspect_values_to_movies,
                               paper_films_aspect_prepended)
    for film, meta, aspects in results:
        aspect_dict[film] = aspects

    return aspect_dict

In [11]:
import numpy as np
book_genres = dict_movie_aspect(info, "genre")
book_authors = dict_movie_aspect(info,'author')
authors = pd.DataFrame.from_dict(book_authors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
genres = pd.DataFrame.from_dict(book_genres,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index

In [12]:
haha_final['authors'] = haha_final['item'].map(info)
haha_final['genres'] = haha_final['item'].map(info)

/Users/user/anaconda3/envs/python37/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/user/anaconda3/envs/python37/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
for i in haha_final.index:
    haha_final['authors'][i] = info[haha_final['item'][i]]['author']
    haha_final['genres'][i] = info[haha_final['item'][i]]['genre']

/Users/user/anaconda3/envs/python37/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/user/anaconda3/envs/python37/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/user/anaconda3/envs/python37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [14]:
haha_final.index = range(len(haha_final))
#del haha_final['url']
jiba = jiba.rename(columns = {0:'real',1:'item'})

In [15]:
final = pd.merge(haha_final,jiba)

In [16]:
del final['item']
final = final.rename(columns = {"real":"item"})

In [17]:
model_df = pd.merge(df,final)

In [18]:
model_df = model_df[['user', 'item', 'authors', 'genres','rating']]

In [19]:
authors_dict = dict(zip(authors, range(len(authors))))
genres_dict = dict(zip(genres, range(
    len(authors), len(authors) + len(genres))))

In [20]:
from operator import itemgetter
def change_author(x):
    qunima = itemgetter(*x)(authors_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return change_list(qunima)
    
def change_genre(x):
    qunima = itemgetter(*x)(genres_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return change_list(qunima)    

def change_list(x):
    return list(x)
hehe = model_df.copy()

In [22]:
hehe['authors'] = hehe['authors'].map(change_author)
hehe['genres'] = hehe['genres'].map(change_genre)

In [23]:
def data_split(df):  # This function randomly splits the whole dataset into training set and test set with the ration train:test = 8:2
    trainset = df.sample(frac=0.8, random_state=0, axis=0)
    trainset.index = range(len(trainset))
    testset = df[~df.index.isin(trainset.index)]
    testset.index = range(len(testset))
    return trainset, testset

In [24]:
#hehe = final.copy()
trainset = hehe.sample(frac=0.8, random_state=0, axis=0)
testset = hehe[~hehe.index.isin(trainset.index)]
trainset.index = range(len(trainset))
testset.index = range(len(testset))
rate = 0.0

In [ ]:
trainset.to_csv('train.csv',index=False)
testset.to_csv('test.csv',index=False)